# Smol model recognize lift

## Load data


In [5]:
import os
import cv2
import random
from sklearn.model_selection import train_test_split

# Step 1: Define paths and parameters
input_folder = 'lift_images'  # Folder containing subfolders "0" and "1"
output_train_folder = 'lift_images/processed_images/train'
output_test_folder = 'lift_images/processed_images/test'
target_size = (240, 426)  # Resize to 240x426
test_size = 0.2  # Percentage of data to use for testing
random_seed = 42  # For reproducibility

# Step 2: Create output directories for train and test sets
os.makedirs(os.path.join(output_train_folder, '0'), exist_ok=True)
os.makedirs(os.path.join(output_train_folder, '1'), exist_ok=True)
os.makedirs(os.path.join(output_test_folder, '0'), exist_ok=True)
os.makedirs(os.path.join(output_test_folder, '1'), exist_ok=True)

# Step 3: Gather all image file paths from both classes
image_paths = []
labels = []

for label in ['0', '1']:
    folder_path = os.path.join(input_folder, label)
    for filename in os.listdir(folder_path):
        if filename.endswith(('.jpg', '.png', '.jpeg')):
            image_paths.append(os.path.join(folder_path, filename))
            labels.append(label)  # Append label corresponding to the folder

# Step 4: Split the data into train and test sets
train_paths, test_paths, train_labels, test_labels = train_test_split(
    image_paths, labels, test_size=test_size, random_state=random_seed
)

# Function to resize and save images
def resize_and_save(image_path, label, output_folder):
    img = cv2.imread(image_path)
    if img is not None:
        resized_img = cv2.resize(img, target_size)
        # Save resized image to the corresponding output folder based on its label
        filename = os.path.basename(image_path)
        output_path = os.path.join(output_folder, label, filename)
        cv2.imwrite(output_path, resized_img)
    else:
        print(f"Error loading image: {image_path}")

# Step 5: Process and save training images
print("Processing training images...")
for img_path, label in zip(train_paths, train_labels):
    resize_and_save(img_path, label, output_train_folder)

# Step 6: Process and save test images
print("Processing test images...")
for img_path, label in zip(test_paths, test_labels):
    resize_and_save(img_path, label, output_test_folder)

print("Image resizing and splitting completed!")


Processing training images...
Processing test images...
Image resizing and splitting completed!


## Model

In [6]:
import os
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Step 1: Create the lightweight CNN model
def create_lightweight_cnn(input_shape=(240, 426, 3), num_classes=2):  # Update num_classes to 2 for binary classification
    model = models.Sequential()
    
    # First convolutional layer (16 filters, kernel size 3x3)
    model.add(layers.Conv2D(16, (3, 3), activation='relu', input_shape=input_shape))
    model.add(layers.MaxPooling2D((2, 2)))  # Max pooling layer

    # Second convolutional layer (32 filters, kernel size 3x3)
    model.add(layers.Conv2D(32, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))  # Max pooling layer

    # Third convolutional layer (32 filters, kernel size 3x3)
    model.add(layers.Conv2D(32, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))  # Max pooling layer

    # Flatten the output to feed into dense layers
    model.add(layers.Flatten())

    # Reduce dense layer size to 32 units
    model.add(layers.Dense(32, activation='relu'))

    # Output layer with softmax for classification (num_classes)
    model.add(layers.Dense(num_classes, activation='softmax'))

    return model


## Training

In [16]:
# Step 2: Set up directories for training and testing images
train_dir = 'lift_images/processed_images/test'
test_dir = 'lift_images/processed_images/train'

# Step 3: Create ImageDataGenerator for loading images
train_datagen = ImageDataGenerator(
    rescale=1.0/255.0,      # Normalize pixel values to 0-1
    rotation_range=20,      # Optional: rotate images for augmentation
    width_shift_range=0.2,  # Optional: shift width for augmentation
    height_shift_range=0.2, # Optional: shift height for augmentation
    shear_range=0.2,        # Optional: shear augmentation
    zoom_range=0.2,         # Optional: zoom augmentation
    horizontal_flip=True,   # Optional: flip images horizontally
    fill_mode='nearest'     # Filling strategy for new pixels after augmentation
)

test_datagen = ImageDataGenerator(rescale=1.0/255.0)  # Only normalization for test data

# Step 4: Load images from directories and apply the transformations
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(240, 426),  # Resize images to the target size
    batch_size=32,           # Number of images per batch
    class_mode='categorical' # Assuming binary classification; change to 'binary' if required
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(240, 426),  # Resize test images
    batch_size=32,           # Batch size for testing
    class_mode='categorical' # Same as for training
)

# Step 5: Compile the model
input_shape = (240, 426, 3)  # Adjust based on image size
num_classes = 2  # Update to 2 for binary classification

model = create_lightweight_cnn(input_shape, num_classes)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Step 6: Summarize the model
model.summary()

# Step 7: Train the model
history = model.fit(
    train_generator,
    epochs=100,                    # Adjust number of epochs as needed
    validation_data=test_generator # Validate with the test data
)

# Step 8: Evaluate the model on the test set
loss, accuracy = model.evaluate(test_generator)
print(f"Test accuracy: {accuracy:.2f}")

model_save_path = 'lift_images/models/lightweight_cnn_model.h5'  # Define the path to save the model
model.save(model_save_path)  # Save the model
print(f"Model saved to: {model_save_path}")


Found 13 images belonging to 2 classes.
Found 52 images belonging to 2 classes.


Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_12 (Conv2D)              │ (None, 238, 424, 16)   │           448 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_12 (MaxPooling2D) │ (None, 119, 212, 16)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_13 (Conv2D)              │ (None, 117, 210, 32)   │         4,640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_13 (MaxPooling2D) │ (None, 58, 105, 32)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_14 (Conv2D)              │ (None, 56, 103, 32)    │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_14 (MaxPooling2D) │ (None, 28, 51, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_4 (Flatten)             │ (None, 45696)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 32)             │     1,462,304 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_9 (Dense)                 │ (None, 2)              │            66 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,476,706 (5.63 MB)

 Trainable params: 1,476,706 (5.63 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 5s 5s/step - accuracy: 0.5385 - loss: 0.6990 - val_accuracy: 0.6154 - val_loss: 2.1399
Epoch 2/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step - accuracy: 0.6154 - loss: 1.9927 - val_accuracy: 0.6154 - val_loss: 0.6600
Epoch 3/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step - accuracy: 0.6154 - loss: 0.7032 - val_accuracy: 0.3846 - val_loss: 0.8384
Epoch 4/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step - accuracy: 0.3846 - loss: 0.8363 - val_accuracy: 0.3846 - val_loss: 0.7327
Epoch 5/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step - accuracy: 0.3846 - loss: 0.7228 - val_accuracy: 0.3846 - val_loss: 0.7578
Epoch 6/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step - accuracy: 0.3846 - loss: 0.7491 - val_accuracy: 0.6154 - val_loss: 0.6715
Epoch 7/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step - accuracy: 0.6154 - loss: 0.6544 - val_accuracy: 0.6154 - val_loss: 0.6836
Epoch 8/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step - accuracy: 0.6154 - loss: 0.7024 - val_accuracy: 0.6154 - val_loss: 0.6891
Epoch 9/

Test accuracy: 0.94
Model saved to: lift_images/models/lightweight_cnn_model.h5


In [19]:
import os
import numpy as np
import cv2
import tensorflow as tf

def load_and_preprocess_image(image_path, target_size=(240, 426)):
    """Load an image, resize it, and normalize pixel values."""
    img = cv2.imread(image_path)
    if img is None:
        raise ValueError(f"Image not found at path: {image_path}")
    img = cv2.resize(img, target_size)  # Resize to the target size
    img = img / 255.0  # Normalize pixel values to [0, 1]
    img = np.expand_dims(img, axis=0)  # Add batch dimension
    return img

def inference(model, test_images_dir):
    """Perform inference on images in the specified directory using the provided model."""
    
    # Get a list of image file paths
    test_image_paths = [os.path.join(test_images_dir, f) for f in os.listdir(test_images_dir) if f.endswith(('.jpg', '.png', '.jpeg'))]

    predictions = []
    
    # Make predictions for each image
    for image_path in test_image_paths:
        processed_image = load_and_preprocess_image(image_path)
        prediction = model.predict(processed_image)  # Get predictions
        predicted_class = np.argmax(prediction, axis=1)  # Convert to class label
        predictions.append((image_path, predicted_class[0]))  # Store image path and predicted class

    return predictions

def print_predictions(predictions):
    """Print the predictions in a readable format."""
    for img_path, pred_class in predictions:
        print(f"Image: {img_path} -> Predicted Class: {pred_class}")

# Example usage
if __name__ == "__main__":
    # Load the trained model (if saved)
    model = tf.keras.models.load_model('lift_images\models\lightweight_cnn_model.h5')  # Uncomment if you saved the model
    
    # Specify the directory containing test images
    test_images_dir = 'lift_images\inference_test'
    
    # Perform inference
    predictions = inference(model, test_images_dir)
    
    # Print the results
    print_predictions(predictions)



<>:41: SyntaxWarning: invalid escape sequence '\m'
<>:44: SyntaxWarning: invalid escape sequence '\i'
<>:41: SyntaxWarning: invalid escape sequence '\m'
<>:44: SyntaxWarning: invalid escape sequence '\i'
C:\Users\tammk\AppData\Local\Temp\ipykernel_19560\3116641692.py:41: SyntaxWarning: invalid escape sequence '\m'
  model = tf.keras.models.load_model('lift_images\models\lightweight_cnn_model.h5')  # Uncomment if you saved the model
C:\Users\tammk\AppData\Local\Temp\ipykernel_19560\3116641692.py:44: SyntaxWarning: invalid escape sequence '\i'
  test_images_dir = 'lift_images\inference_test'


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 190ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
Image: lift_images\inference_test\PXL_20241022_111038390.jpg -> Predicted Class: 0
Image: lift_images\inference_test\PXL_20241022_111041093.jpg -> Predicted Class: 1
Image: lift_images\inference_test\PXL_20241022_114943950.jpg -> Predicted Class: 0


## Convert to tflite


In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Save the model
with open("model.tflite", "wb") as f:
    f.write(tflite_model)